<a href="https://colab.research.google.com/github/LeeDongN/DN_Python/blob/master/Readme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Find Tumor  

---

##해결해야 할 것
1. FindCoordinates

2. Tumor_pre-processing

    - tumor_img_pre-processing
    - tumor_mask_pre-processing

---

###1. FindCoordinates
####접근 방법:
    
    - mask img는 정확히 흑, 백(grayscare)으로 나눠져 있으니 픽셀단위로 접근하여 최소직사각형을 구하자.

   - **opencv에서 제공하는 contours, hierarchy 함수를 사용함.**



In [0]:
def find_coordinates(DATA_KIND, SAVE_PATH):
    
    coordinates_list = []
    #이미지에서 좌표를 찾는다
    for a in DATA_KIND:
        
        img_binary = cv.imread(a, 0)
        contours, hierarchy = cv.findContours(img_binary, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
        basename = os.path.basename(a)
        
        for cnt in contours:
            x, y, w, h = cv.boundingRect(cnt)
        #[(x0, y0), (x1, y1)] 형태로 저장

        line = []
        #왼쪽 아래, 오른쪽 위 좌표 저장
        line.append(basename)
        line.append(str(x))
        line.append(str(y + h))
        line.append(str(x + w))
        line.append(str(y))
                  
        coordinates_list.append(line)   
     
    generate_text(SAVE_PATH, coordinates_list,  '\n')

- find_coordinates 함수는 mask이미지의 최소사각형의 **왼쪽 아래 점과 오른쪽 위의 점**을 리스트에 저장한다. 

In [0]:
def generate_text(SAVE_PATH, each_coordinates_list,  sep):
    
    DATA_basename = ['tif', 'png', 'bmp']
    
    global i
    file = open(SAVE_PATH + 'DATA_{}'.format(DATA_basename[i]) + '.txt', 'w')
        
    for a in range(len(each_coordinates_list)): 
            
        vstr = ''
        vstr = vstr + str(each_coordinates_list[a])+ sep         
        vstr = vstr.rstrip(sep)
           
        file.write(vstr)
        file.write("\n")
        file.write("\n")
        a += 1
        
    file.close()
       
    i += 1

- generate_text 함수는 find_coordinates 함수 이후에 호출되며, find_coordinates에서 coordinates_list를 반환받아 파일에 저장한다.

- DATA_basename는 확장자명이다.


---

###2. tumor_img_pre-processing

###접근방법:

 - raw_image에서 잘라야 할 부분의 경계가 명확하다. 따라서 blur과 Canny 함수를 이용하여 잘라야 할 부위를 파악한다.

- **1단계: 노이즈 제거(Noise Reduction)**

         이미지에서 노이즈가 있으면 edge를 제대로 찾기 어렵다. 따라서 가우시안 필터를 이용하여 이미지의 노이즈를 줄여준다.

- **2단계: Gradient 값이 높은 부분 찾기**

          가우시안 필터로 노이즈가 제거된 이미지를 Canny함수를 이용하여 edge를 검출한다.

In [0]:
def img_cutting(DATA_KIND, SAVE_PATH):

    for a in DATA_KIND:

        img_color = cv.imread(a)
        img_gray = cv.imread(a, cv.IMREAD_GRAYSCALE)
        basename = os.path.basename(a)
        
        #이미지의 윤곽선을 딴다
        blur = cv.GaussianBlur(img_gray, ksize = (5, 5), sigmaX = 0)
        ret, thresh1 = cv.threshold(blur, 127, 255, cv.THRESH_BINARY)

        edged = cv.Canny(blur, 10, 250)


        contours, _ = cv.findContours(edged.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


        contours_xy = np.array(contours)
        contours_xy.shape

        #이미지의 x, y의 최대 최소 값을 구한
        x_min, x_min = 0, 0
        value = list()
        
        for i in range(len(contours_xy)):
            for j in range(len(contours_xy[i])):
                value.append(contours_xy[i][j][0][0])
                x_min = min(value)
                x_max = max(value)


        y_min, y_min = 0, 0
        value = list()
        
        for i in range(len(contours_xy)):
            for j in range(len(contours_xy[i])):
                value.append(contours_xy[i][j][0][1])
                y_min = min(value)
                y_max = max(value)


        x = x_min
        y = y_min
        w = x_max - x_min
        h = y_max - y_min
        
        img_trim = img_color[y:y+h, x:x+w]
        cv.imwrite(SAVE_PATH + basename, img_trim)

- img_cutting 함수는 이미지의 경계 값을 찾고, 그 중 **최소, 최대의 (x, y)값을 검출한 뒤 이미지를 자른다.

---

##tumor_mask_pre-processing

###접근방법:

- tumor_img_pre-processing 함수에서 도출된 x, y, w, h를 받아 mask를 자른다.

In [0]:
def mask_cutting(x, y, w, h, SAVE_PATH_mask, basename, TYPE):
    
    #data유형에 따라 (_mask + TYPE)를 바꿔줘야 함. 
    mask_gray = cv.imread(path_mask + basename[:-4] + '_mask'  + TYPE, cv.IMREAD_GRAYSCALE)
        
    mask_trim = mask_gray[y:y+h, x:x+w]
    cv.imwrite(SAVE_PATH_mask + basename[:-4] + '_mask' + TYPE, mask_trim)
    print(basename[:-4] + '_mask' + TYPE)

- TYPE는 확장자명(.png, .bmp, .tif)이다.

- basename[:-4]는 확장자명을 제외한 파일 이름을 불러오기 위함.